In [3]:
import numpy as np
from PIL import ImageGrab
import cv2
import time
import pyautogui
from numpy import ones,vstack
from numpy.linalg import lstsq
from directkeys import PressKey,ReleaseKey, W, A, S, D
from statistics import mean

def roi(img, vertices):
    
    #blank mask:
    mask = np.zeros_like(img)   
    
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, 255)
    
    #returning the image only where mask pixels are nonzero
    masked = cv2.bitwise_and(img, mask)
    return masked


def draw_lanes(img, lines, color=[0, 255, 255], thickness=3):

    # if this fails, go with some default line
    try:

        # finds the maximum y value for a lane marker 
        # (since we cannot assume the horizon will always be at the same point.)

        ys = []  
        for i in lines:
            for ii in i:
                ys += [ii[1],ii[3]]
        min_y = min(ys)
        max_y = 600
        new_lines = []
        line_dict = {}

        for idx,i in enumerate(lines):
            for xyxy in i:
                # These four lines:
                # modified from http://stackoverflow.com/questions/21565994/method-to-return-the-equation-of-a-straight-line-given-two-points
                # Used to calculate the definition of a line, given two sets of coords.
                x_coords = (xyxy[0],xyxy[2])
                y_coords = (xyxy[1],xyxy[3])
                A = vstack([x_coords,ones(len(x_coords))]).T
                m, b = lstsq(A, y_coords)[0]

                # Calculating our new, and improved, xs
                x1 = (min_y-b) / m
                x2 = (max_y-b) / m

                line_dict[idx] = [m,b,[int(x1), min_y, int(x2), max_y]]
                new_lines.append([int(x1), min_y, int(x2), max_y])

        final_lanes = {}

        for idx in line_dict:
            final_lanes_copy = final_lanes.copy()
            m = line_dict[idx][0]
            b = line_dict[idx][1]
            line = line_dict[idx][2]
            
            if len(final_lanes) == 0:
                final_lanes[m] = [ [m,b,line] ]
                
            else:
                found_copy = False

                for other_ms in final_lanes_copy:

                    if not found_copy:
                        if abs(other_ms*1.2) > abs(m) > abs(other_ms*0.8):
                            if abs(final_lanes_copy[other_ms][0][1]*1.2) > abs(b) > abs(final_lanes_copy[other_ms][0][1]*0.8):
                                final_lanes[other_ms].append([m,b,line])
                                found_copy = True
                                break
                        else:
                            final_lanes[m] = [ [m,b,line] ]

        line_counter = {}

        for lanes in final_lanes:
            line_counter[lanes] = len(final_lanes[lanes])

        top_lanes = sorted(line_counter.items(), key=lambda item: item[1])[::-1][:2]

        lane1_id = top_lanes[0][0]
        lane2_id = top_lanes[1][0]

        def average_lane(lane_data):
            x1s = []
            y1s = []
            x2s = []
            y2s = []
            for data in lane_data:
                x1s.append(data[2][0])
                y1s.append(data[2][1])
                x2s.append(data[2][2])
                y2s.append(data[2][3])
            return int(mean(x1s)), int(mean(y1s)), int(mean(x2s)), int(mean(y2s)) 

        l1_x1, l1_y1, l1_x2, l1_y2 = average_lane(final_lanes[lane1_id])
        l2_x1, l2_y1, l2_x2, l2_y2 = average_lane(final_lanes[lane2_id])

        return [l1_x1, l1_y1, l1_x2, l1_y2], [l2_x1, l2_y1, l2_x2, l2_y2], lane1_id, lane2_id
    except Exception as e:
        print(str(e))


def process_img(image):
    original_image = image
    # edge detection
    processed_img =  cv2.Canny(image, threshold1 = 200, threshold2=300)
    
    processed_img = cv2.GaussianBlur(processed_img,(5,5),0)
    
    vertices = np.array([[10,500],[10,300],[300,200],[500,200],[800,300],[800,500],
                         ], np.int32)

    processed_img = roi(processed_img, [vertices])

    # more info: http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_houghlines/py_houghlines.html
    #                                     rho   theta   thresh  min length, max gap:        
    lines = cv2.HoughLinesP(processed_img, 1, np.pi/180, 180,      20,       15)
    m1 = 0
    m2 = 0
    try:
        l1, l2, m1,m2 = draw_lanes(original_image,lines)
        cv2.line(original_image, (l1[0], l1[1]), (l1[2], l1[3]), [0,255,0], 30)
        cv2.line(original_image, (l2[0], l2[1]), (l2[2], l2[3]), [0,255,0], 30)
    except Exception as e:
        print(str(e))
        pass
    try:
        for coords in lines:
            coords = coords[0]
            try:
                cv2.line(processed_img, (coords[0], coords[1]), (coords[2], coords[3]), [255,0,0], 3)
                
                
            except Exception as e:
                print(str(e))
    except Exception as e:
        pass

    return processed_img,original_image, m1, m2

def straight():
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)

def left():
    PressKey(A)
    ReleaseKey(W)
    ReleaseKey(D)
    ReleaseKey(A)

def right():
    PressKey(D)
    ReleaseKey(A)
    ReleaseKey(W)
    ReleaseKey(D)

def slow_ya_roll():
    ReleaseKey(W)
    ReleaseKey(A)
    ReleaseKey(D)


for i in list(range(4))[::-1]:
    print(i+1)
    time.sleep(1)


last_time = time.time()
while True:
    screen =  np.array(ImageGrab.grab(bbox=(0,40,800,640)))
    print('Frame took {} seconds'.format(time.time()-last_time))
    last_time = time.time()
    new_screen,original_image, m1, m2 = process_img(screen)
    #cv2.imshow('window', new_screen)
    cv2.imshow('window2',cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))



    if m1 < 0 and m2 < 0:
        right()
    elif m1 > 0  and m2 > 0:
        left()
    else:
        straight()
    
    #cv2.imshow('window',cv2.cvtColor(screen, cv2.COLOR_BGR2RGB))
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break



4
3
2
1
Frame took 0.11170172691345215 seconds


e:\program\anaconda\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


Frame took 0.7056319713592529 seconds
Frame took 0.12865209579467773 seconds
Frame took 0.12224793434143066 seconds
Frame took 0.12666034698486328 seconds
Frame took 0.11967897415161133 seconds
Frame took 0.1316978931427002 seconds
Frame took 0.1296389102935791 seconds
Frame took 0.11666369438171387 seconds
Frame took 0.12166285514831543 seconds
Frame took 0.1296532154083252 seconds
Frame took 0.11593985557556152 seconds
Frame took 0.11719441413879395 seconds
Frame took 0.11922407150268555 seconds
Frame took 0.1267080307006836 seconds
Frame took 0.11865091323852539 seconds
Frame took 0.11570096015930176 seconds
Frame took 0.11916494369506836 seconds
Frame took 0.12973904609680176 seconds
Frame took 0.12185144424438477 seconds
Frame took 0.10965776443481445 seconds
Frame took 0.12113142013549805 seconds
Frame took 0.11969161033630371 seconds
Frame took 0.1336383819580078 seconds
Frame took 0.12668728828430176 seconds
Frame took 0.12064337730407715 seconds
Frame took 0.1136941909790039 s

Frame took 0.14962053298950195 seconds
Frame took 0.13463926315307617 seconds
Frame took 0.13419818878173828 seconds
Frame took 0.1498887538909912 seconds
Frame took 0.13218331336975098 seconds
Frame took 0.15062832832336426 seconds
Frame took 0.13065671920776367 seconds
Frame took 0.1338191032409668 seconds
Frame took 0.14979839324951172 seconds
Frame took 0.14879679679870605 seconds
Frame took 0.13683843612670898 seconds
Frame took 0.14776349067687988 seconds
Frame took 0.14957046508789062 seconds
Frame took 0.1346731185913086 seconds
Frame took 0.13586664199829102 seconds
Frame took 0.14504694938659668 seconds
Frame took 0.135606050491333 seconds
Frame took 0.13164710998535156 seconds
Frame took 0.13458895683288574 seconds
Frame took 0.1316814422607422 seconds
Frame took 0.13468408584594727 seconds
Frame took 0.13278818130493164 seconds
Frame took 0.13489627838134766 seconds
Frame took 0.13165020942687988 seconds
Frame took 0.13605690002441406 seconds
Frame took 0.12968659400939941 

Frame took 0.12064599990844727 seconds
Frame took 0.11369538307189941 seconds
Frame took 0.11868715286254883 seconds
Frame took 0.1137228012084961 seconds
Frame took 0.11823391914367676 seconds
Frame took 0.1141672134399414 seconds
Frame took 0.11968493461608887 seconds
Frame took 0.12956786155700684 seconds
Frame took 0.11873269081115723 seconds
Frame took 0.11360621452331543 seconds
Frame took 0.119476318359375 seconds
Frame took 0.11686420440673828 seconds
Frame took 0.1147615909576416 seconds
Frame took 0.11483883857727051 seconds
Frame took 0.11771678924560547 seconds
Frame took 0.11665821075439453 seconds
Frame took 0.1216726303100586 seconds
Frame took 0.10874128341674805 seconds
Frame took 0.12064671516418457 seconds
Frame took 0.11411190032958984 seconds
Frame took 0.12064647674560547 seconds
Frame took 0.11469364166259766 seconds
Frame took 0.11672282218933105 seconds
Frame took 0.11695313453674316 seconds
Frame took 0.11586523056030273 seconds
Frame took 0.11963796615600586 

Frame took 0.14860844612121582 seconds
Frame took 0.15261125564575195 seconds
Frame took 0.14612960815429688 seconds
Frame took 0.15561127662658691 seconds
Frame took 0.14561152458190918 seconds
Frame took 0.14859867095947266 seconds
Frame took 0.15059709548950195 seconds
Frame took 0.14763593673706055 seconds
Frame took 0.15555214881896973 seconds
Frame took 0.14484524726867676 seconds
Frame took 0.1495985984802246 seconds
Frame took 0.1488020420074463 seconds
Frame took 0.1521754264831543 seconds
Frame took 0.1501152515411377 seconds
Frame took 0.14757061004638672 seconds
Frame took 0.15113234519958496 seconds
Frame took 0.1486053466796875 seconds
Frame took 0.15159153938293457 seconds
Frame took 0.1655876636505127 seconds
Frame took 0.1515951156616211 seconds
Frame took 0.14957165718078613 seconds
Frame took 0.16755247116088867 seconds
Frame took 0.1495966911315918 seconds
Frame took 0.16307973861694336 seconds
Frame took 0.1511671543121338 seconds
Frame took 0.14942526817321777 sec

KeyboardInterrupt: 